#### Trained and Deployed a Price range predictor model on aws

1. S3 Buckets- Boto3
2. Iam Roles and Users
3. Complete Infrastructure of AWS Sagemaker-Training, Endpoints

In [34]:
! pip install dotenv


In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

AWS_REGION = os.environ.get("AWS_REGION", "us-east-1")

AWS_ACCESS_KEY_ID=os.environ.get("AWS_ACCESS_KEY_ID","")
AWS_SECRET_ACCESS_KEY=os.environ.get("AWS_SECRET_ACCESS_KEY","")



In [ ]:
! pip install sagemaker scikit-learn pandas numpy  ipykernel

In [26]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd




boto_sess = boto3.session.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=None,
    region_name=AWS_REGION # if False else "ca-central-1"
)

sm_boto3=boto3.client("sagemaker", aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=None,
    region_name=AWS_REGION )

sess=sagemaker.Session(boto_session=boto_sess)
region=sess.boto_session.region_name
bucket="pricesagemaker213"
print("Using bucket" + bucket)


Using bucketpricesagemaker213


In [3]:
print(region)

us-east-1


In [4]:
df=pd.read_csv("/mob_price_classification_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
df.shape

(2000, 21)

In [6]:
df.isnull().sum()

,0
battery_power,0
blue,0
clock_speed,0
dual_sim,0
fc,0
four_g,0
int_memory,0
m_dep,0
mobile_wt,0
n_cores,0


In [7]:
df['price_range'].value_counts()

,count
price_range,
1,500
2,500
3,500
0,500


In [8]:
features=list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [9]:
label = features.pop(-1)
label

'price_range'

In [10]:
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi']

In [12]:
x=df[features]
y=df[label]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [15]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [16]:
trainX

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1224,1,1.6,0,9,0,33,1.0,157,1,...,522,563,3796,10,5,13,1,1,0,3
1216,1158,0,0.7,1,1,1,29,0.7,123,2,...,311,1796,1542,17,9,15,1,0,1,1
1653,1190,0,2.0,1,0,0,40,0.2,93,5,...,1399,1646,3610,13,7,9,0,0,1,3
559,1191,0,2.4,1,2,0,13,0.9,169,1,...,179,1813,1028,14,6,8,1,1,1,0


In [17]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [18]:
bucket

'pricesagemaker213'

In [32]:
## send data to S3. Sagemaker will take the data for trainign from s3
sk_prefix="sagemaker/mobile_price_classification/sklearncontainer"
trainpath=sess.upload_data(path='train-V-1.csv',bucket=bucket,key_prefix=sk_prefix)

testpath=sess.upload_data(path='test-V-1.csv',bucket=bucket,key_prefix=sk_prefix)

print(trainpath)
print(testpath)

s3://pricesagemaker213/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://pricesagemaker213/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


#### Script used by AWS Sagemaker To Train Models

In [19]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd


def model_fn(model_dir):
    clf=joblib.load(os.path.join(model_dir,"model.joblib"))

if __name__=="__main__":

    print("[Info] Extracting arguments")
    parser=argparse.ArgumentParser()

    ## Hyperparameter
    parser.add_argument("--n_estimators",type=int,default=100)
    parser.add_argument("--random_state",type=int,default=0)

    ### Data,model,and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ =parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()

    print("Label column is: ",label)
    print()

    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()


    print("Training RandomForest Model ....")
    print()
    model=RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,
                                 verbose=2,n_jobs=1)

    model.fit(X_train,y_train)

    print()

    model_path=os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)

    print("Model saved at" + model_path)

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)




Overwriting script.py


### AWS Sagemaker Entry Point To Execute the Training script

In [20]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION="0.23-1"

sklearn_estimator=SKLearn(
    entry_point="script.py",
    role="", # the aws sagemaker role goes here
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    sagemaker_session=sess,
    hyperparameters={
        "n_estimators":100,
        "random_state":0
    },
    use_spot_instance=True,
    max_run=900,                      # Hard cap on training seconds (here 15 min). Caps cost.
    #max_wait=1800,                    # Required with Spot; gives time to find capacity.
    volume_size=10,                   # EBS GB for /opt/ml, billed per GB-month; smaller = cheaper (10 GB is fine).
    enable_sagemaker_metrics=False

)

In [67]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

2025-11-12 21:20:28 Starting - Starting the training job...
2025-11-12 21:20:42 Starting - Preparing the instances for training...
2025-11-12 21:21:29 Downloading - Downloading the training image......
2025-11-12 21:22:25 Training - Training image download completed. Training in progress.
2025-11-12 21:22:25 Uploading - Uploading generated training model2025-11-12 21:22:19,204 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-11-12 21:22:19,207 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-11-12 21:22:19,253 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-11-12 21:22:19,439 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-11-12 21:22:19,452 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-11-12 21:22:19,463 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-

### To get the model from S3

In [ ]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

In [ ]:
artifact

### Deploy the Model For Endpoint

In [30]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime,strftime


model_name=str("Custom-sklearn-model-0" + strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
model=SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::761018869540:role/sagemakerole",
    entry_point="script.py",
    sagemaker_session=sess,
    framework_version=FRAMEWORK_VERSION
)

In [31]:
model

In [32]:
## Endpoint deployment
endpoint_name = f"Custom-sklearn-model-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
print("EndpointName={}".format(endpoint_name))
import logging, sagemaker
logging.getLogger("sagemaker").setLevel(logging.INFO)
# now do your model.deploy(...)

predictor=model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name

)

EndpointName=Custom-sklearn-model-2025-11-12-22-21-49


INFO:sagemaker:Creating model with name: Custom-sklearn-model-0
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2025-11-12-22-21-49
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2025-11-12-22-21-49


-------!

In [33]:
predictor

In [34]:
testX[features][0:2]

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
405,1454,1,0.5,1,1,0,34,0.7,83,4,3,250,1033,3419,7,5,5,1,1,0
1190,1092,1,0.5,1,10,0,11,0.5,167,3,14,468,571,737,14,4,11,0,1,0


In [ ]:
#make a prediction and display the results
print(predictor.predict(testX[features][:2].values.tolist()))

In [ ]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)